In [ ]:
# This code is a study code, aiming to make new parameter sets and pass new look-up table and function 

In [1]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import timeit
start = timeit.default_timer()


In [3]:
# Define a function to calculate concentration change!
def Cal_new_con_for_Li_EC(Sol,Para,c_EC_0,c_e_0):
    #############################################################################################################################  
    ###############################           Step-1 Prepare parameters:        #################################################
    #############################################################################################################################  
    L_p   =Para["Positive electrode thickness [m]"]
    L_n   =Para["Negative electrode thickness [m]"]
    L_s   =Para["Separator thickness [m]"]
    L_y   =Para["Electrode width [m]"]   # Also update to change A_cc and therefore Q
    L_z   =Para["Electrode height [m]"]
    # Initial EC concentration  
    Ini_Ce_EC        =Para["Bulk solvent concentration [mol.m-3]"]  # Be careful when multiplying with pore volume to get amount in mole. Because of electrolyte dry out, that will give more amount than real.   
    # LLI due to electrode,Ratio of EC and lithium is 1:1 -> EC amount consumed is LLINegSEI[-1]
    LLINegSEI = Sol["Loss of lithium to SEI [mol]"].entries[-1] - Sol["Loss of lithium to SEI [mol]"].entries[0]
    # Pore volume change with time:
    PoreVolNeg = Sol["X-averaged negative electrode porosity"].entries*L_n*L_y*L_z;
    PoreVolSep = Sol["X-averaged separator porosity"].entries*L_s*L_y*L_z;
    PoreVolPos = Sol["X-averaged positive electrode porosity"].entries*L_p*L_y*L_z;
    #############################################################################################################################  
    #####################       Step-2 Determine How much electrolyte is added, whether dry out:        #########################
    #############################################################################################################################  
    Vol_Elely_Tot_old = Para["Current total electrolyte volume in whole cell [m3]"] 
    Vol_Elely_JR_old  = Para["Current total electrolyte volume in jelly roll [m3]"] 
    Vol_Pore_tot_old  = PoreVolNeg[0] + PoreVolSep[0] + PoreVolPos[0]       # pore volume at start time of the run
    Vol_Pore_tot_new  = PoreVolNeg[-1] + PoreVolSep[-1] + PoreVolPos[-1]    # pore volume at end   time of the run, intrinsic variable 
    Vol_Pore_decrease = Vol_Pore_tot_old  - Vol_Pore_tot_new;
    # EC:lithium:SEI=2:2:1     for SEI=(CH2OCO2Li)2
    # Because inner and outer SEI partial molar volume is the same, just set one for whole SEI
    VmolSEI   = Para["Outer SEI partial molar volume [m3.mol-1]"] # 9.8e-5,
    VmolEC    = 6.667e-5  # Unit:m3.mol-1; According to Wiki, correct value: EC molar volume is :66.67 cm3.mol-1  = 6.667e-5, 
    Vol_EC_consumed  = LLINegSEI * VmolEC * 2
    Vol_Elely_need   = Vol_EC_consumed - Vol_Pore_decrease
    Vol_SEI_increase = 0.5*LLINegSEI*VmolSEI   #  volume increase due to SEI
    Test_V = Vol_SEI_increase - Vol_Pore_decrease  #  This value should always be zero, but now not, which becomes the source of error!
    # Important: Calculate added electrolyte based on excessive electrolyte, can be: 1) added as required; 2) added some, but less than required; 3) added 0 
    if Vol_Elely_Tot_old > Vol_Elely_JR_old:                             # This means Vol_Elely_JR_old = Vol_Pore_tot_old ()
        if Vol_Elely_Tot_old-Vol_Elely_JR_old >= Vol_Elely_need:         # 1) added as required
            Vol_Elely_add     = Vol_Elely_need;
            Vol_Elely_Tot_new = Vol_Elely_Tot_old -  Vol_EC_consumed;
            Vol_Elely_JR_new  = Vol_Elely_JR_old  -  Vol_Pore_decrease;  # also equals to 'Vol_Pore_tot_new', or Vol_Pore_tot_old - Vol_Pore_decrease
            Ratio_Dryout = 1.0;
        else:                                                            # 2) added some, but less than required;                                                         
            Vol_Elely_add     = Vol_Elely_Tot_old - Vol_Elely_JR_old;   
            Vol_Elely_Tot_new = Vol_Elely_Tot_old - Vol_EC_consumed;
            Vol_Elely_JR_new  = Vol_Elely_Tot_new;                       # This means Vol_Elely_JR_new <= Vol_Pore_tot_new
            Ratio_Dryout = Vol_Elely_JR_new/Vol_Pore_tot_new;
    else:                                                                # 3) added 0 
        Vol_Elely_add = 0;
        Vol_Elely_Tot_new = Vol_Elely_Tot_old - Vol_EC_consumed;
        Vol_Elely_JR_new  = Vol_Elely_Tot_new; 
        Ratio_Dryout = Vol_Elely_JR_new/Vol_Pore_tot_new;
    #############################################################################################################################  
    ###################       Step-3 Electrolyte mixing, define ratio change for Li and EC concentration        #################
    #############################################################################################################################     
    # Next: start mix electrolyte based on previous defined equation
    # Lithium amount in liquid phase, at initial time point
    TotLi_Elely_JR_Old = Sol["Total lithium in electrolyte [mol]"].entries[-1];
    # Added lithium and EC amount in the added lithium electrolyte: - should always be really 'initial'
    AddLiMol   =  Vol_Elely_add*c_e_0
    AddECMol   =  Vol_Elely_add*c_EC_0
    # Total amount of Li and EC in current electrolyte:
    # Remember Li has two parts, initial and added; EC has three parts, initial, consumed and added
    TotLi_Elely_JR_New   = TotLi_Elely_JR_Old + AddLiMol
    TotECMol_JR   = Vol_Elely_JR_old*Ini_Ce_EC - LLINegSEI*2 + AddECMol
    Ratio_CeEC  = TotECMol_JR    /   Vol_Elely_JR_new   / Ini_Ce_EC
    Ratio_CeLi  = TotLi_Elely_JR_New    /   TotLi_Elely_JR_Old   /  Ratio_Dryout # Mark, change on 21-11-19
    Width_new   = Ratio_Dryout * L_y;
    # Collect the above parameter in Data_Pack to shorten the code  
    Data_Pack   = [Vol_EC_consumed, Vol_Elely_need, Test_V, Vol_Elely_add, Vol_Elely_Tot_new, Vol_Elely_JR_new, Vol_Pore_tot_new, Vol_Pore_decrease, Ratio_Dryout, Ratio_CeEC, Ratio_CeLi,Width_new, ]  # 11 in total
    return Data_Pack

In [4]:
# Define a function to update parameter and lithium concentration in solution     
def Update_param ( Para, Ratio_CeEC, Ratio_Dryout, Vol_Elely_Tot, Vol_Elely_JR ):
    Old_CeEC = Para["Bulk solvent concentration [mol.m-3]"]
    Old_L_y  = Para["Electrode width [m]"]
    Para.update(   {'Bulk solvent concentration [mol.m-3]':  Old_CeEC * Ratio_CeEC  })
    Para.update(   {'Current total electrolyte volume in whole cell [m3]':  Vol_Elely_Tot  })
    Para.update(   {'Current total electrolyte volume in jelly roll [m3]':  Vol_Elely_JR  })
    Para.update(   {'Ratio of electrolyte dry out in jelly roll':Ratio_Dryout})
    Para.update(   {'Electrode width [m]':Ratio_Dryout * Old_L_y})
    # Elely_Con_Dimless_old = Sol["Electrolyte concentration"].entries 
    # Sol.update(      {'Electrolyte concentration': Elely_Con_Dimless_old*Ratio_CeLi  })  # Important: update solution - turn out not work
    return Para   # call function: Para_new, Sol_new = Update_param ( Sol, Para, Ratio_CeLi, Ratio_CeEC, Ratio_Dryout )
# Para_update, Sol_update  = Update_param (  Sol_All_0[-1], Para_All[-1], Ratio_CeLi_All[-1], Ratio_CeEC_All[-1], Ratio_Dryout_All[-1]  )

# Define a function to calculate based on previous solution
def Run_Model_Base_On_Last_Solution( Model  , Sol, Ratio_CeLi, Para_update, ModelExperiment, Update_Cycles ):
    # Use Sulzer's method: inplace = false
    ModelTimer = pb.Timer()
    # Important line: define new model based on previous solution
    dict_short = {}; 
    list_short = [  "Discharge capacity [A.h]",
        "Negative particle concentration",
        "Positive particle concentration",
        "Negative electrode potential",
        "Positive electrode potential",
        "Total negative electrode interfacial current density variable" ,
        "Total positive electrode interfacial current density variable",
        "Negative electrode porosity times concentration",
        "Separator porosity times concentration",
        "Positive electrode porosity times concentration",
        "Inner SEI thickness",
        "Outer SEI thickness",
        "Negative electrolyte potential",
        "Separator electrolyte potential",
        "Positive electrolyte potential",   ]
    for list_short_i in list_short:
        dict_short.update( { list_short_i : Sol.last_state[list_short_i].data  }  )
    dict_short["Negative electrode porosity times concentration"] = dict_short["Negative electrode porosity times concentration"] * Ratio_CeLi # important: update sol here!
    dict_short["Separator porosity times concentration"] = dict_short["Separator porosity times concentration"] * Ratio_CeLi # important: update sol here!
    dict_short["Positive electrode porosity times concentration"] = dict_short["Positive electrode porosity times concentration"] * Ratio_CeLi # important: update sol here!
    ModelModel_new = Model.set_initial_conditions_from(dict_short, inplace=False)
    Para_update.update(   {'Ambient temperature [K]':313.15 });
    Para_update.update(   {'Initial temperature [K]':313.15 });
    ModelSimnew = pb.Simulation(
        ModelModel_new,
        experiment = ModelExperiment, 
        parameter_values=Para_update, 
        solver = pb.CasadiSolver(),
    )
    Sol_new = ModelSimnew.solve(save_at_cycles = Update_Cycles) # save every several cycles, can save RAM greatly
    # print("Solved this model in {}".format(ModelTimer.time()))
    return ModelModel_new, Sol_new
def Run_Model_Base_On_Last_Solution_RPT( Model  , Sol, Ratio_CeLi,  Para_update, ModelExperiment ,Update_Cycles):
    # Use Sulzer's method: inplace = false
    ModelTimer = pb.Timer()
    # print("Model is now using average EC Concentration of:",Para_update['Bulk solvent concentration [mol.m-3]'])
    # print("Ratio of electrolyte dry out in jelly roll is:",Para_update['Ratio of electrolyte dry out in jelly roll'])
    # print("Model is now using an electrode width of:",Para_update['Electrode width [m]'])
    # Important line: define new model based on previous solution
    dict_short = {}; 
    list_short = [  "Discharge capacity [A.h]",
        "Negative particle concentration",
        "Positive particle concentration",
        "Negative electrode potential",
        "Positive electrode potential",
        "Total negative electrode interfacial current density variable" ,
        "Total positive electrode interfacial current density variable",
        "Negative electrode porosity times concentration",
        "Separator porosity times concentration",
        "Positive electrode porosity times concentration",
        "Inner SEI thickness",
        "Outer SEI thickness",
        "Negative electrolyte potential",
        "Separator electrolyte potential",
        "Positive electrolyte potential",   ]
    for list_short_i in list_short:
        dict_short.update( { list_short_i : Sol.last_state[list_short_i].data  }  )
    dict_short["Negative electrode porosity times concentration"] = dict_short["Negative electrode porosity times concentration"] * Ratio_CeLi # important: update sol here!
    dict_short["Separator porosity times concentration"]          = dict_short["Separator porosity times concentration"] * Ratio_CeLi          # important: update sol here!
    dict_short["Positive electrode porosity times concentration"] = dict_short["Positive electrode porosity times concentration"] * Ratio_CeLi # important: update sol here!
    ModelModel_new = Model.set_initial_conditions_from(dict_short, inplace=False)
    Para_update.update(   {'Ambient temperature [K]':298.15 });
    Para_update.update(   {'Initial temperature [K]':298.15 });
    ModelSimnew = pb.Simulation(
        ModelModel_new,
        experiment = ModelExperiment, 
        parameter_values=Para_update, 
        solver = pb.CasadiSolver(),
    )
    Sol_new = ModelSimnew.solve(save_at_cycles = Update_Cycles) # save every several cycles, can save RAM greatly
    # print("Solved this model in {}".format(ModelTimer.time()))
    return ModelModel_new, Sol_new

In [5]:
# Define electrolyte diffusivity and conductivity
def electrolyte_diffusivity_Nyman2008(c_e, T):
    if c_e < 2000:
        D_c_e = 8.794e-11 * (c_e / 1000) ** 2 - 3.972e-10 * (c_e / 1000) + 4.862e-10
    else:
        D_c_e = 8.794e-11 * (2000 / 1000) ** 2 - 3.972e-10 * (2000 / 1000) + 4.862e-10
    return D_c_e    
def electrolyte_conductivity_Nyman2008(c_e, T):
    if c_e < 2000:
        sigma_e = (0.1297 * (c_e / 1000) ** 3 - 2.51 * (c_e / 1000) ** 1.5 + 3.329 * (c_e / 1000))
    else:
        sigma_e = (0.1297 * (2000 / 1000) ** 3 - 2.51 * (2000 / 1000) ** 1.5 + 3.329 * (2000 / 1000))
    return sigma_e

In [6]:
pb.parameter_sets.Chen2020_coupled

{'chemistry': 'lithium-ion',
 'cell': 'LGM50_Chen2020_coupled',
 'negative electrode': 'graphite_Chen2020_coupled',
 'separator': 'separator_Chen2020',
 'positive electrode': 'nmc_Chen2020_coupled',
 'electrolyte': 'lipf6_Nyman2008_coupled',
 'experiment': '1C_discharge_from_full_Chen2020',
 'sei': 'example_coupled'}

In [7]:
path = os.getcwd()
print(path)

d:\GitPyBaMM\LRH_409\LRH_409\PyBaMM-GEM\wip\Rio_Code\P2_R0\office


In [8]:
os.chdir(pb.__path__[0]+'\\input\\parameters')    #os.chdir(pb.__path__[0]+'/..')
path = os.getcwd()
print(path)

d:\gitpybamm\lrh_409\lrh_409\pybamm-gem\pybamm\input\parameters


In [2]:
pb.PARAMETER_PATH

['e:\\allwork\\python\\pybamm\\gitcode_pybamm\\pybamm-gem',
 'e:\\allwork\\Python\\Pybamm\\GitCode_PyBaMM\\PyBaMM-GEM\\wip\\Rio_Code\\P2_R0\\office',
 'e:\\allwork\\python\\pybamm\\gitcode_pybamm\\pybamm-gem\\pybamm\\input\\parameters']

In [12]:
%%bash
cp -r pybamm/input/parameters/lithium_ion/negative_electrodes/graphite_Chen2020/ pybamm/input/parameters/lithium_ion/negative_electrodes/graphite_workshop

In [13]:
text_exp = [("Discharge at 1C for 0.5 hours", "Discharge at C/20 for 0.5 hours")] * 3 + [("Charge at 0.5 C for 45 minutes",)]
Experiment_Pre_Ageing  = pb.Experiment([("Discharge at 1C for 0.5 hours", "Discharge at C/20 for 0.5 hours")] * 3 + [("Charge at 0.5 C for 45 minutes",)])

In [14]:
# Write Niall's experiment:
V_min = 2.5;  V_max = 4.2;  Update_Cycles = 10; RPT_Cycles = 1; 
Experiment_Pre_Ageing  = pb.Experiment( [    # under 15,25,35 degreeC
    (
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 4 hours",  
    )  ])  

Experiment_Ageing  = pb.Experiment( [         # under 15,25,35 degreeC
    (f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/100")    
    ] * 77 )     # can be changed for update cycles

Experiment_RPT_Long    = pb.Experiment( [      # under 25 degreeC
    (
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
        f"Discharge at 0.1 C until {V_min} V",  
        "Rest for 6 hours",  
        f"Charge at 0.1 C until {V_max} V",
        "Rest for 10 minutes",  
    ) ] * 1    # Sub-1: C/10 CC
    +
    [(
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
        f"Discharge at 0.5 C until {V_min} V",  
        "Rest for 6 hours",  
        f"Charge at 0.5 C until {V_max} V",
        "Rest for 30 minutes",  
    ) ] * 1    # Sub-2: C/2 CC
    +
    [(
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
    ) ] * 1    # Sub-3-1: Prepare for C/2 GITT
    +
    [(
        f"Discharge at 0.5 C for 4.8 minutes or until {V_min} V",                 
        "Rest for 1 hour",  
    ) ] * 24    # Sub-3-2: Do C/2 GITT - 25 pulses            # how to set jump condition for voltage?
    +
    [(
        "Rest for 5 hours",  
    ) ] * 1    # Sub-3-3: Post C/2 GITT
    +
    [(
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
    ) ] * 1    # Sub-4-1: Prepare for C/2 GITT
    +
    [(
        f"Discharge at 0.5 C for 24 minutes or until {V_min} V",                  
        "Rest for 1 hour",  
    ) ] * 4    # Sub-4-2: Do C/2 GITT - 25 pulses            # how to set jump condition for voltage?
    +
    [(
        "Rest for 5 hours",  
    )  * 1    # Sub-4-3: Post C/2 GITT
    ] ) 

Experiment_RPT_Short    = pb.Experiment( [ 
    (
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
        f"Discharge at 0.1 C until {V_min} V",  
        "Rest for 6 hours",  
        f"Charge at 0.1 C until {V_max} V",
        "Rest for 10 minutes",  
    ) ] * 1    # Sub-1: C/10 CC
    +
    [(
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
    ) ] * 1    # Sub-2-1: Prepare for Pulse under load
    +
    [(
        f"Discharge at 0.5 C for 30 seconds or until {V_min} V",                   
        f"Discharge at 0.55 C for 10 seconds or until {V_min} V",   
        f"Discharge at 0.5 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.45 C for 10 seconds or until {V_min} V",   
        f"Discharge at 0.5 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.625 C for 10 seconds or until {V_min} V",   
        f"Discharge at 0.5 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.375 C for 10 seconds or until {V_min} V", 
        f"Discharge at 0.5 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.75 C for 10 seconds or until {V_min} V", 
        f"Discharge at 0.5 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.25 C for 10 seconds or until {V_min} V", 
    )  ]* 29    # Sub-2-2: Do Pulse under load (C/2 average current, discharge)
    +
    [(
        "Rest for 5 hours",  
    ) ] * 1 
    +
    [(
        f"Charge at 0.5 C for 30 seconds or until {V_max} V",                   
        f"Charge at 0.55 C for 10 seconds or until {V_max} V",   
        f"Charge at 0.5 C for 30 seconds or until {V_max} V",   
        f"Charge at 0.45 C for 10 seconds or until {V_max} V",   
        f"Charge at 0.5 C for 30 seconds or until {V_max} V",   
        f"Charge at 0.625 C for 10 seconds or until {V_max} V",   
        f"Charge at 0.5 C for 30 seconds or until {V_max} V",   
        f"Charge at 0.375 C for 10 seconds or until {V_max} V", 
        f"Charge at 0.5 C for 30 seconds or until {V_max} V",   
        f"Charge at 0.75 C for 10 seconds or until {V_max} V", 
        f"Charge at 0.5 C for 30 seconds or until {V_max} V",   
        f"Charge at 0.25 C for 10 seconds or until {V_max} V", 
    ) ] * 29    # Sub-2-3: Do Pulse under load (C/2 average current, charge)
    +
    [(
        "Rest for 10 minutes",  
    ) ] * 1 
    +
    [(
        f"Charge at 0.3 C until {V_max} V", 
        f"Hold at {V_max} V until C/100",
        "Rest for 2 hours",  
    ) ] * 1    # Sub-3-1: Prepare for Pulse under load
    +
    [(
        f"Discharge at 1 C for 30 seconds or until {V_min} V",                   
        f"Discharge at 1.1 C for 10 seconds or until {V_min} V",   
        f"Discharge at 1 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.9 C for 10 seconds or until {V_min} V",   
        f"Discharge at 1 C for 30 seconds or until {V_min} V",   
        f"Discharge at 1.25 C for 10 seconds or until {V_min} V",   
        f"Discharge at 1 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.75 C for 10 seconds or until {V_min} V", 
        f"Discharge at 1 C for 30 seconds or until {V_min} V",   
        f"Discharge at 1.5 C for 10 seconds or until {V_min} V", 
        f"Discharge at 1 C for 30 seconds or until {V_min} V",   
        f"Discharge at 0.5 C for 10 seconds or until {V_min} V", 
    ) ] * 14    # Sub-3-2: Do Pulse under load (C/2 average current, discharge)
    +
    [(
        "Rest for 6 hours",  
    )  * 1 
    ] ) 

Experiment_Breakin= pb.Experiment( [    # at 25 degreeC
    (f"Discharge at 0.2 C until {V_min} V",
    "Rest for 4 hours",
    f"Charge at 0.2 C until {V_max} V" , 
    f"Hold at {V_max} V until C/100",
    "Rest for 2 hours")        ]   *5
    )  


In [7]:
# Global setting Niall's result:
Total_Cycles = 1170; Cycle_bt_RPT = 78; Update_Cycles = 1;  Small_Loop =  int(Cycle_bt_RPT/Update_Cycles);   SaveTimes = int(Total_Cycles/Cycle_bt_RPT);   RPT_Cycles = 1; 

# Special cases for Non-dry-out model:
Total_Cycles_NonDry = 1170-78;   SaveTimes_NonDry = int(Total_Cycles_NonDry/Cycle_bt_RPT);  

Real_SaveAs_Cycles = 1 # Set this to use it in real experiment so that can really get capacity throughput!
V_max = 4.2;        V_min = 2.5;
cs_Neg_Init = 28920; Diff_SEI = 1.700000000000002e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pb.Experiment( [ (f"Discharge at 1 C until {V_min} V", f"Charge at 0.3 C until {V_max} V", f"Hold at {V_max} V until C/100")    ] * Update_Cycles )  
Experiment_RPT    = pb.Experiment( [ (f"Discharge at 0.1 C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1 C until {V_max} V" ) ] * RPT_Cycles ) 
Experiment_Breakin= pb.Experiment( [ (f"Discharge at 0.2 C until {V_min} V","Rest for 4 hours",f"Charge at 0.2 C until {V_max} V" , f"Hold at {V_max} V until C/100","Rest for 2 hours") *5 + ( f"Discharge at 0.1 C until {V_min} V",  "Rest for 6 hours",  f"Charge at 0.1 C until {V_max} V" ) *1 ] )
BasicPath = 'E:/allwork/Python/Pybamm/Study/';
Target = 'Test R0/';   
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);
################ Important: index definition #################################
index = list(np.arange(1,SaveTimes+1)*(Small_Loop+RPT_Cycles)-1); #index.insert(0,0)  # get all last ageing cycle before RPT, plus the first RPT test
index2= list(np.arange(0,SaveTimes+1)*(Small_Loop+RPT_Cycles));         # get all RPT test
cycles = np.arange(0,SaveTimes+1)*Cycle_bt_RPT; 
cycles3 = list(np.arange(1,SaveTimes+1)*(Cycle_bt_RPT)); 

index_NonDry    = list(np.arange(1,SaveTimes_NonDry+1)*(Small_Loop+RPT_Cycles)-1); #index.insert(0,0)  # get all last ageing cycle before RPT, plus the first RPT test
index2_NonDry   = list(np.arange(0,SaveTimes_NonDry+1)*(Small_Loop+RPT_Cycles));         # get all RPT test  
cycles_NonDry   = np.arange     (0,SaveTimes_NonDry+1)*Cycle_bt_RPT; 
cycles3_NonDry  = list(np.arange(1,SaveTimes_NonDry+1)*(Cycle_bt_RPT)); 

In [8]:
Model_0 = pb.lithium_ion.DFN(
    options={
        "particle": "Fickian diffusion", 
        "SEI":"interstitial-diffusion limited", 
        "SEI film resistance":"distributed", 
        "SEI porosity change":"true",
        "lithium plating":"partially reversible"
    }
)

In [9]:
chemistry = pb.parameter_sets.Chen2020_coupled
Para_0 = pb.ParameterValues(chemistry=chemistry)
#Para_0["Ambient temperature [K]"] = pybamm.InputParameter("T")
Para_0.update({"Upper voltage cut-off [V]": 4.20})
Para_0.update({"Lower voltage cut-off [V]": 2.50})
Para_0.update({'SEI resistivity [Ohm.m]':R_SEI}) ;
Para_0.update({'Outer SEI solvent diffusivity [m2.s-1]':Diff_SEI});  
Para_0.update({'Bulk solvent concentration [mol.m-3]':Bulk_Sol_Con});
Para_0.update({'Initial concentration in negative electrode [mol.m-3]':cs_Neg_Init }) # 
Para_0.update({"Lithium plating kinetic rate constant [m.s-1]": 1E-18})
Para_0.update({"SEI-plating coupling constant [s-1]": 2E-6})

In [10]:
Sol_NonDry=[];  Model_NonDry=[];
Sim_0    = pb.Simulation(Model_0, experiment = Experiment_Breakin,parameter_values = Para_0,solver = pb.CasadiSolver(),)       
Sol_0    = Sim_0.solve(save_at_cycles=RPT_Cycles);
Sol_NonDry.append(Sol_0); Model_NonDry.append(Model_0);